In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
sns.set(style="white", context='paper')

In [82]:
import os
dpath = '..'
os.listdir(dpath)

['.ipynb_checkpoints',
 'cds_from_genomic.fna',
 'GCF_001578515.1_ASM157851v1_genomic.fna',
 'GCF_001578515.1_ASM157851v1_genomic.fna.fai',
 'genomic.gbff',
 'genomic.gff',
 'genomic.gtf',
 'HOT1A3 gbff to EC list.ipynb',
 'HOT1A3 genomic.gff',
 'HOT1A3.bed',
 'HOT1A3_gff_to_csv.ipynb',
 'HOT1A3_KO.tsv',
 'HOT1A3_ncbi_double_protein_id.csv',
 'HOT1A3_NCBI_ECnumber.tsv',
 'HOT1A3_NCBI_gene_list.csv',
 'HOT1A3_operons_2643684.tar.gz',
 'HOT1A3_pathways.csv',
 'HOT1A3_pathways_category.csv',
 'HOT1A3_protein.faa',
 'HOT1A3_protein.tsv',
 'HOT1A3_protein_list.csv',
 'HOT1A3_reactions_biocyc.tst',
 'HOT1A3_reactions_biocyc_multi.txt',
 'HOT1A3_user_ko.txt',
 'kegg_map_pathway2category.xlsx',
 'manage genome HOT1A3.ipynb',
 'MIT1002_ATCC27126_HOT1A3_id-lookup.csv',
 'sequence_report.jsonl',
 'signalp_ROS_protease']

In [90]:
#df = pd.read_csv('10cc_ecpath_master.csv')
ec_df = pd.read_csv('../HOT1A3_NCBI_ECnumber.tsv', sep='\t', names=['locus_tag', 'EC_number'])
ec_df


,locus_tag,EC_number
0,ACZ81_RS00010,2.7.7.7
1,ACZ81_RS00020,5.6.2.2
2,ACZ81_RS00030,6.1.1.14
3,ACZ81_RS00035,6.1.1.14
4,ACZ81_RS00070,2.7.7.7
...,...,...
755,ACZ81_RS19870,7.1.2.2
756,ACZ81_RS19890,2.1.1.170
757,ACZ81_RS19900,2.7.7.49
758,ACZ81_RS19920,3.6.-.-


In [91]:
signal_df = pd.read_csv('data/HOT1A3_signalP.csv')

In [92]:
ko_df = pd.read_csv('../HOT1A3_protein_list.csv', )
#ko_df

In [93]:
signal_df.protein_id.value_counts()
signal_map = signal_df.Prediction
signal_map.index = signal_df.protein_id
ko_df['SignalP'] = ko_df.protein_id.map(signal_map)
ko_df['is_signal'] = ko_df.SignalP.isin(['OTHER']).map({True: 'Not Signal', False:'Is Signal'})

In [94]:
ko_df['EC_number_from_kegg'] = ko_df.ecpath.str.extract(r'\[EC:(.*)\]')

In [95]:
# TODO - there's multiple EC per locus_tag
#ko_df2 = pd.merge(ko_df, ec_df, how='left', on='locus_tag')

In [96]:
ko_df2 = ko_df[['locus_tag', 'EC_number_from_kegg']].copy()
ko_df2 = ko_df2.assign(EC_number_from_kegg=ko_df2.EC_number_from_kegg.str.split(" ")).explode("EC_number_from_kegg")

# look for ecnumbers

In [97]:
second_selected_ecs =  pd.read_excel(os.path.join('data','N related  enzymes.xlsx'))

In [98]:
second_selected_ecs

,category,N Enzyme name,EC numbers,Unnamed: 3
0,Membrane transporters,Membrane transporters,3.6.3.x,NaN
1,Membrane transporters,Polar-amino-acid-transporting ATPase,3.6.3.21,NaN
2,Membrane transporters,Nonpolar-amino-acid-transporting ATPase,3.6.3.22,NaN
3,Membrane transporters,Oligopeptide-transporting ATPase,3.6.3.23,NaN
4,Membrane transporters,Nitrate-transporting ATPase,3.6.3.26,NaN
...,...,...,...,...
73,Other enzymes relevant to N cycling,Citrate synthase,2.3.3.8,NaN
74,Other enzymes relevant to N cycling,Carbon–nitrogen lyases,4.3.x.x,NaN
75,Other enzymes relevant to N cycling,Forming carbon–nitrogen bonds,6.3.x.x,NaN
76,Other enzymes relevant to N cycling,"Acting on carbon-nitrogen bonds, other than pe...",3.5.x.x,NaN


In [99]:
third_selected_kos =  pd.read_excel(os.path.join('data','ROS related  enzymes.xlsx',))


In [100]:
second_selected_ecs['EC_number'] = second_selected_ecs['EC numbers'].str.replace('x', '-', regex=False)

In [101]:
second_selected_ecs.EC_number.str.count(r'\-').value_counts()

EC_number
0    59
1    16
2     3
Name: count, dtype: int64

In [103]:
ec_df.EC_number.str.count(r'\-').value_counts()

EC_number
0    686
1     61
2     12
3      1
Name: count, dtype: int64

In [104]:
ko_df2.EC_number_from_kegg.str.count(r'\-').value_counts()

EC_number_from_kegg
0.0    1163
1.0     105
2.0      15
3.0       6
Name: count, dtype: int64

In [105]:
ec_df['EC_number1'] = ec_df['EC_number'].str.replace(r'(\d+)$', '-', regex=True)
ec_df['EC_number2'] = ec_df['EC_number1'].str.replace(r'(\d+)\.\-$', '-.-', regex=True)
ec_df[['EC_number', 'EC_number1', 'EC_number2']].value_counts(dropna=False).reset_index()

,EC_number,EC_number1,EC_number2,count
0,2.7.7.49,2.7.7.-,2.7.-.-,16
1,2.7.7.65,2.7.7.-,2.7.-.-,16
2,2.7.7.7,2.7.7.-,2.7.-.-,10
3,7.1.2.2,7.1.2.-,7.1.-.-,9
4,2.4.-.-,2.4.-.-,2.4.-.-,8
...,...,...,...,...
568,2.5.1.6,2.5.1.-,2.5.-.-,1
569,2.5.1.55,2.5.1.-,2.5.-.-,1
570,2.5.1.47,2.5.1.-,2.5.-.-,1
571,2.5.1.39,2.5.1.-,2.5.-.-,1


In [106]:
ko_df2['EC_number_from_kegg1'] = ko_df2['EC_number_from_kegg'].str.replace(r'(\d+)$', '-', regex=True)
ko_df2['EC_number_from_kegg2'] = ko_df2['EC_number_from_kegg1'].str.replace(r'(\d+)\.\-$', '-.-', regex=True)
ko_df2[['EC_number_from_kegg', 'EC_number_from_kegg1', 'EC_number_from_kegg2']].value_counts(dropna=False).reset_index()

,EC_number_from_kegg,EC_number_from_kegg1,EC_number_from_kegg2,count
0,NaN,NaN,NaN,2861
1,2.7.13.3,2.7.13.-,2.7.-.-,30
2,2.7.7.49,2.7.7.-,2.7.-.-,19
3,2.7.7.7,2.7.7.-,2.7.-.-,13
4,5.6.2.4,5.6.2.-,5.6.-.-,11
...,...,...,...,...
888,2.6.1.11,2.6.1.-,2.6.-.-,1
889,2.6.1.112,2.6.1.-,2.6.-.-,1
890,2.6.1.17,2.6.1.-,2.6.-.-,1
891,2.6.1.2,2.6.1.-,2.6.-.-,1


In [107]:
ec_df2_merge1 = pd.merge(ec_df, second_selected_ecs, 
         on=['EC_number'], how='left', indicator=True, suffixes=['', '_pattern'])
matched_ids1 = ec_df2_merge1.loc[~ec_df2_merge1['EC numbers'].isna(), 'locus_tag'].dropna()
matched_ids1


ec_df2_merge2 = pd.merge(ec_df.loc[~ec_df.locus_tag.isin(matched_ids1)], second_selected_ecs, 
         left_on=['EC_number1'], right_on=['EC_number'], how='left', indicator=True,  suffixes=['', '_pattern'])
matched_ids2 = ec_df2_merge2.loc[~ec_df2_merge2['EC numbers'].isna(), 'locus_tag'].dropna()
matched_ids2


ec_df2_merge3 = pd.merge(ec_df.loc[~ec_df.locus_tag.isin(matched_ids1) & ~ec_df.locus_tag.isin(matched_ids2)], second_selected_ecs, 
         left_on=['EC_number2'], right_on=['EC_number'], how='left', indicator=True,  suffixes=['', '_pattern'])
matched_ids3 = ec_df2_merge3.loc[~ec_df2_merge3['EC numbers'].isna(), 'locus_tag'].dropna()
matched_ids3


ec_df3 = pd.concat([
    ec_df2_merge1.loc[ec_df2_merge1['locus_tag'].isin(matched_ids1)],                        
    ec_df2_merge2.loc[ec_df2_merge1['locus_tag'].isin(matched_ids2)],                        
    ec_df2_merge3,
], ignore_index=True)



In [108]:
kec_df2_merge1 = pd.merge(ko_df2, second_selected_ecs, 
         left_on=['EC_number_from_kegg'], right_on=['EC_number'], how='left', indicator=True, suffixes=['', '_pattern'])
matched_ids1 = kec_df2_merge1.loc[~kec_df2_merge1['EC numbers'].isna(), 'locus_tag'].dropna()
matched_ids1


kec_df2_merge2 = pd.merge(ko_df2.loc[~ko_df2.locus_tag.isin(matched_ids1)], second_selected_ecs, 
         left_on=['EC_number_from_kegg1'], right_on=['EC_number'], how='left', indicator=True,  suffixes=['', '_pattern'])
matched_ids2 = kec_df2_merge2.loc[~kec_df2_merge2['EC numbers'].isna(), 'locus_tag'].dropna()
matched_ids2


kec_df2_merge3 = pd.merge(ko_df2.loc[~ko_df2.locus_tag.isin(matched_ids1) & ~ko_df2.locus_tag.isin(matched_ids2)], second_selected_ecs, 
         left_on=['EC_number_from_kegg2'], right_on=['EC_number'], how='left', indicator=True,  suffixes=['', '_pattern'])
matched_ids3 = kec_df2_merge3.loc[~kec_df2_merge3['EC numbers'].isna(), 'locus_tag'].dropna()
matched_ids3


kec_df3 = pd.concat([
    kec_df2_merge1.loc[kec_df2_merge1['locus_tag'].isin(matched_ids1)],                        
    kec_df2_merge2.loc[kec_df2_merge1['locus_tag'].isin(matched_ids2)],                        
    kec_df2_merge3,
], ignore_index=True)



In [109]:
ec_df3

,locus_tag,EC_number,EC_number1,EC_number2,category,N Enzyme name,EC numbers,Unnamed: 3,_merge,EC_number_pattern
0,ACZ81_RS01160,3.4.24.-,3.4.24.-,3.4.-.-,Proteases,Metalloendopeptidases,3.4.24.x,NaN,both,NaN
1,ACZ81_RS01175,3.4.24.-,3.4.24.-,3.4.-.-,Proteases,Metalloendopeptidases,3.4.24.x,NaN,both,NaN
2,ACZ81_RS04740,1.1.1.41,1.1.1.-,1.1.-.-,Other enzymes relevant to N cycling,Isocitrate dehydrogenase,1.1.1.41,(NAD),both,NaN
3,ACZ81_RS04865,3.4.24.-,3.4.24.-,3.4.-.-,Proteases,Metalloendopeptidases,3.4.24.x,NaN,both,NaN
4,ACZ81_RS09200,1.1.1.42,1.1.1.-,1.1.-.-,Other enzymes relevant to N cycling,Isocitrate dehydrogenase,1.1.1.42,(NADP),both,NaN
...,...,...,...,...,...,...,...,...,...,...
755,ACZ81_RS19870,7.1.2.2,7.1.2.-,7.1.-.-,NaN,NaN,NaN,NaN,left_only,NaN
756,ACZ81_RS19890,2.1.1.170,2.1.1.-,2.1.-.-,NaN,NaN,NaN,NaN,left_only,NaN
757,ACZ81_RS19900,2.7.7.49,2.7.7.-,2.7.-.-,NaN,NaN,NaN,NaN,left_only,NaN
758,ACZ81_RS19920,3.6.-.-,3.6.-.-,3.6.-.-,NaN,NaN,NaN,NaN,left_only,NaN


In [111]:
kec_df3

,locus_tag,EC_number_from_kegg,EC_number_from_kegg1,EC_number_from_kegg2,category,N Enzyme name,EC numbers,Unnamed: 3,EC_number,_merge
0,ACZ81_RS03805,3.4.21.-,3.4.21.-,3.4.-.-,Proteases,Serine endopeptidases,3.4.21.x,NaN,3.4.21.-,both
1,ACZ81_RS03810,3.4.21.-,3.4.21.-,3.4.-.-,Proteases,Serine endopeptidases,3.4.21.x,NaN,3.4.21.-,both
2,ACZ81_RS04740,1.1.1.41,1.1.1.-,1.1.-.-,Other enzymes relevant to N cycling,Isocitrate dehydrogenase,1.1.1.41,(NAD),1.1.1.41,both
3,ACZ81_RS04865,3.4.24.-,3.4.24.-,3.4.-.-,Proteases,Metalloendopeptidases,3.4.24.x,NaN,3.4.24.-,both
4,ACZ81_RS05375,3.4.24.-,3.4.24.-,3.4.-.-,Proteases,Metalloendopeptidases,3.4.24.x,NaN,3.4.24.-,both
...,...,...,...,...,...,...,...,...,...,...
4145,ACZ81_RS20690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4146,ACZ81_RS21705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4147,ACZ81_RS20705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4148,ACZ81_RS20720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [110]:
ec_df3.loc[ec_df3._merge.isin(['both'])][['EC_number', 'EC numbers']].value_counts(dropna=False)

EC_number  EC numbers
3.4.24.-   3.4.24.x      5
3.5.4.4    3.5.x.x       3
3.4.21.-   3.4.21.x      3
6.3.5.5    6.3.x.x       2
3.5.1.18   3.5.x.x       2
                        ..
3.5.4.5    3.5.x.x       1
1.1.1.42   1.1.1.42      1
4.3.1.17   4.3.x.x       1
4.3.1.19   4.3.x.x       1
3.5.4.9    3.5.x.x       1
Name: count, Length: 77, dtype: int64

In [112]:
kec_df3.loc[kec_df3._merge.isin(['both'])][['EC_number', 'EC numbers']].value_counts(dropna=False)

EC_number  EC numbers
3.5.-.-    3.5.x.x       56
6.3.-.-    6.3.x.x       36
4.3.-.-    4.3.x.x       11
3.4.21.-   3.4.21.x       9
3.4.24.-   3.4.24.x       7
2.6.1.-    2.6.1.x        3
3.4.16.-   3.4.16.x       2
3.4.19.-   3.4.19.x       2
1.4.1.13   1.4.1.13       2
3.6.3.-    3.6.3.x        2
3.4.23.-   3.4.23.x       1
1.1.1.41   1.1.1.41       1
1.1.1.42   1.1.1.42       1
3.4.15.-   3.4.15.x       1
3.4.13.-   3.4.13.x       1
2.3.3.1    2.3.3.1        1
1.4.3.16   1.4.3.16       1
1.4.1.4    1.4.1.4        1
1.4.1.2    1.4.1.2        1
6.3.1.2    6.3.1.2        1
Name: count, dtype: int64

In [113]:
ko_df2 = pd.merge(ko_df, third_selected_kos[['KO','category', 'enzyme name']], 
         left_on='path_id', right_on='KO', how='left')

In [114]:
ko_df2[['category', 'enzyme name']].value_counts()

category  enzyme name                   
ROS       catalase                          3
          glutathione peroxidase            2
          alkyl hydroperoxide reductase     1
          catalase-peroxidase               1
          cytochrome c peroxidase           1
          glutaredoxin 3                    1
          monothiol glutaredoxin            1
          nickel superoxide dismutase       1
          superoxide dismutase 1            1
          superoxide dismutase 2            1
          thiol peroxidase (tpx)            1
          thioredoxin 1                     1
          thioredoxin reductase             1
Name: count, dtype: int64

In [115]:
ko_df2[['category', ]].value_counts()

category
ROS         16
Name: count, dtype: int64

In [116]:
ec_df3.drop_duplicates(['locus_tag', 'category',])[['category', 'N Enzyme name']].value_counts()


category                             N Enzyme name                                            
Other enzymes relevant to N cycling  Acting on carbon-nitrogen bonds, other than peptide bonds    36
                                     Forming carbon–nitrogen bonds                                31
                                     Carbon–nitrogen lyases                                        9
Proteases                            Metalloendopeptidases                                         5
                                     Serine endopeptidases                                         5
Other enzymes relevant to N cycling  Isocitrate dehydrogenase                                      2
Proteases                            Cysteine endopeptidases                                       2
D-/L-Amino acid oxidases             L-Aspartate oxidase                                           1
Proteases                            Aspartic endopeptidases                                     

In [117]:
kec_df3.drop_duplicates(['locus_tag', 'category',])[['category', 'N Enzyme name']].value_counts()


category                             N Enzyme name                                            
Other enzymes relevant to N cycling  Acting on carbon-nitrogen bonds, other than peptide bonds    56
                                     Forming carbon–nitrogen bonds                                35
                                     Carbon–nitrogen lyases                                       11
Proteases                            Serine endopeptidases                                         9
                                     Metalloendopeptidases                                         7
                                     Serine carboxypeptidases                                      2
Uptake, reduction and assimilation   Glutamate synthase                                            2
Other enzymes relevant to N cycling  Isocitrate dehydrogenase                                      2
Uptake, reduction and assimilation   Glutamate dehydrogenase                                     

In [118]:
ec_df3.drop_duplicates(['locus_tag', 'category'])[['category', ]].value_counts()


category                           
Other enzymes relevant to N cycling    78
Proteases                              13
Uptake, reduction and assimilation      3
D-/L-Amino acid oxidases                1
Name: count, dtype: int64

In [119]:
kec_df3.drop_duplicates(['locus_tag', 'category'])[['category', ]].value_counts()


category                           
Other enzymes relevant to N cycling    105
Proteases                               23
Uptake, reduction and assimilation       5
Membrane transporters                    2
Transaminases                            2
D-/L-Amino acid oxidases                 1
Name: count, dtype: int64

In [120]:
ec_df3.loc[ec_df3.locus_tag.duplicated(keep=False)& ~ec_df3.category.isna()]

,locus_tag,EC_number,EC_number1,EC_number2,category,N Enzyme name,EC numbers,Unnamed: 3,_merge,EC_number_pattern
59,ACZ81_RS00235,6.3.2.5,6.3.2.-,6.3.-.-,Other enzymes relevant to N cycling,Forming carbon–nitrogen bonds,6.3.x.x,NaN,both,6.3.-.-
487,ACZ81_RS12280,6.3.2.12,6.3.2.-,6.3.-.-,Other enzymes relevant to N cycling,Forming carbon–nitrogen bonds,6.3.x.x,NaN,both,6.3.-.-
488,ACZ81_RS12280,6.3.2.17,6.3.2.-,6.3.-.-,Other enzymes relevant to N cycling,Forming carbon–nitrogen bonds,6.3.x.x,NaN,both,6.3.-.-
505,ACZ81_RS12740,3.5.4.19,3.5.4.-,3.5.-.-,Other enzymes relevant to N cycling,"Acting on carbon-nitrogen bonds, other than pe...",3.5.x.x,NaN,both,3.5.-.-
522,ACZ81_RS13040,3.5.4.26,3.5.4.-,3.5.-.-,Other enzymes relevant to N cycling,"Acting on carbon-nitrogen bonds, other than pe...",3.5.x.x,NaN,both,3.5.-.-
537,ACZ81_RS13385,3.5.4.9,3.5.4.-,3.5.-.-,Other enzymes relevant to N cycling,"Acting on carbon-nitrogen bonds, other than pe...",3.5.x.x,NaN,both,3.5.-.-
690,ACZ81_RS18105,3.5.2.3,3.5.2.-,3.5.-.-,Other enzymes relevant to N cycling,"Acting on carbon-nitrogen bonds, other than pe...",3.5.x.x,NaN,both,3.5.-.-
723,ACZ81_RS19200,3.5.4.10,3.5.4.-,3.5.-.-,Other enzymes relevant to N cycling,"Acting on carbon-nitrogen bonds, other than pe...",3.5.x.x,NaN,both,3.5.-.-


In [121]:
ko_df.locus_tag.value_counts()
ko_df2.loc[ko_df2.locus_tag.duplicated(keep=False)& ~ko_df2.category.isna()]


,seq_id,Name,gene,gene_synonym,locus_tag,old_locus_tag,source,start,end,strand,...,Category,Membrane transport,kegg_gene1,kegg_gene2,SignalP,is_signal,EC_number_from_kegg,KO,category,enzyme name


In [28]:
ec_df3[ec_df3['category'].isin(['Proteases'])].drop_duplicates('locus_tag')

,locus_tag,EC_number,EC_number1,EC_number2,category,N Enzyme name,EC numbers,Unnamed: 3,_merge,EC_number_pattern
0,ACZ81_RS01160,3.4.24.-,3.4.24.-,3.4.-.-,Proteases,Metalloendopeptidases,3.4.24.x,NaN,both,NaN
1,ACZ81_RS01175,3.4.24.-,3.4.24.-,3.4.-.-,Proteases,Metalloendopeptidases,3.4.24.x,NaN,both,NaN
3,ACZ81_RS04865,3.4.24.-,3.4.24.-,3.4.-.-,Proteases,Metalloendopeptidases,3.4.24.x,NaN,both,NaN
5,ACZ81_RS09380,3.4.24.-,3.4.24.-,3.4.-.-,Proteases,Metalloendopeptidases,3.4.24.x,NaN,both,NaN
7,ACZ81_RS09910,3.4.21.-,3.4.21.-,3.4.-.-,Proteases,Serine endopeptidases,3.4.21.x,NaN,both,NaN
8,ACZ81_RS10780,3.4.24.-,3.4.24.-,3.4.-.-,Proteases,Metalloendopeptidases,3.4.24.x,NaN,both,NaN
9,ACZ81_RS11025,3.4.21.-,3.4.21.-,3.4.-.-,Proteases,Serine endopeptidases,3.4.21.x,NaN,both,NaN
11,ACZ81_RS13865,3.4.22.-,3.4.22.-,3.4.-.-,Proteases,Cysteine endopeptidases,3.4.22.x,NaN,both,NaN
12,ACZ81_RS14275,3.4.22.-,3.4.22.-,3.4.-.-,Proteases,Cysteine endopeptidases,3.4.22.x,NaN,both,NaN
13,ACZ81_RS14675,3.4.23.-,3.4.23.-,3.4.-.-,Proteases,Aspartic endopeptidases,3.4.23.x,NaN,both,NaN


In [ ]:
stop

In [38]:
ec_df4['is_signal'] = ec_df4.SignalP.isin(['OTHER']).map({True: 'Not Signal', False:'Is Signal'})

In [39]:
ec_df4[[ 'category', 'is_signal' ]].value_counts(dropna=False).reset_index()

,category,is_signal,count
0,NaN,Not Signal,4028
1,NaN,Is Signal,586
2,Other enzymes relevant to N cycling,Not Signal,239
3,Proteases,Not Signal,34
4,Proteases,Is Signal,28
5,Other enzymes relevant to N cycling,Is Signal,26
6,Membrane transporters,Not Signal,25
7,"Uptake, reduction and assimilation",Not Signal,19
8,ROS,Not Signal,18
9,Transaminases,Not Signal,15


In [133]:
protease_ids = set(ec_df3.loc[ec_df3.category.isin(['Proteases', 'D-/L-Amino acid oxidases'])].locus_tag.unique()) | \
    set(kec_df3.loc[kec_df3.category.isin(['Proteases', 'D-/L-Amino acid oxidases'])].locus_tag.unique())

In [134]:
ROS_ids = ko_df2.loc[ko_df2.category.isin(['ROS'])].locus_tag.unique()


In [131]:
ko_df.shape, ko_df2.shape

((4023, 28), (4023, 31))

In [132]:
ko_df2.category.value_counts()

category
ROS    16
Name: count, dtype: int64

In [135]:
ko_df['ROS_degradation'] = 'Not ROS degradation'
ko_df.loc[ko_df.locus_tag.isin(ROS_ids), 'ROS_degradation'] = 'ROS_degradation'

In [136]:
ko_df['Protease_Aminase'] = 'Not Protease/Aminase'
ko_df.loc[ko_df.locus_tag.isin(protease_ids), 'Protease_Aminase'] = 'Protease/Aminase'

In [143]:
ko_df[['Protease_Aminase', 'is_signal']].value_counts()


Protease_Aminase      is_signal 
Not Protease/Aminase  Not Signal    3280
                      Is Signal      710
Protease/Aminase      Not Signal      19
                      Is Signal       14
Name: count, dtype: int64

In [144]:
ko_df[['ROS_degradation', 'is_signal']].value_counts()


ROS_degradation      is_signal 
Not ROS degradation  Not Signal    3286
                     Is Signal      721
ROS_degradation      Not Signal      13
                     Is Signal        3
Name: count, dtype: int64

In [142]:
ko_df.to_csv('HOT1A3_ROS_protease.csv', index=False)